# K-NN 알고리즘으로  붓꽃 분류하기

### # 라이브러리 설치하기

In [ ]:
!pip install scikit-learn 

### # Scikit-learn 사용하기

In [ ]:
from sklearn.datasets import load_iris
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split

-----------------------

## 1.K-NN 알고리즘의 원리
- 1. 분류하고 싶은 데이터와 분류된 데이터 사이의 거리 계산하기
- 2. 분류하고 싶은 데이터와 가까운 순서대로 나열하기
- 3. 가장 가까운 K개 데이터 중에서 레이블별 빈도 세기
- 4. return 최다 빈도의 붓꽃 레이블값 변환하기

### [실습] 두 점사이의 거리 구하기 : 유클리드 거리법

In [ ]:
import numpy as np

# 두 점 A(1,2)와 B(5,7) 선언
A = np.array([1, 2])
B = np.array([5, 7])

# 두 점 A(1,2)와 B(5,7) 사이의 거리를 계산하여 변수 distance에 저장하기
distance = np.sqrt(np.sum(np.power((A-B), 2)))

# 변수 distance에 저장된 값을 소수점 셋째 자리에서 반올림하기
distance = round(distance, 2)

# 계산 결과 출력하기
print(f'두 점 A(1,2)와 B(5,7) 사이의 거리는 {distance} 입니다.')

### [실습] 붓꽃 데이터 그래프로 나타내기

* kaggle 사이트에서 데이터 다운로드하기 
    - https://www.kaggle.com/uciml/iris?select=Iris.csv
    - 또는 UCI 사이트에서 데이터 다운로드 하기
    - https://archive.ics.uci.edu/ml/index.php

#### 1. 데이터 가져오기

In [ ]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

matplotlib.rcParams['font.family'] = 'Malgun Gothic' # 맑은 고딕'으로 설정  
plt.rcParams["figure.figsize"] = (10,6)         # 그래프 크기 설정

# 데이터 가져오기
df = pd.read_csv('data/Iris.csv')    
df

#### # 각 붓꽃의 종류별 데이터 추출하기

In [ ]:
# 각 붓꽃의 종류별 데이터 추출하기
setosa    = df.query(" Species == 'Iris-setosa' ") # df[df.Species=='Iris-setosa'] 
versicolor= df.query(" Species == 'Iris-versicolor' ")
virginica = df.query(" Species == 'Iris-virginica' ")

#### 2. 산포도(scatter) 그래프 그리기
 - 꽃받침(Sepal)의 길이와 너비에 따른 산포도 그래프

In [ ]:
# 꽃받침(Sepal)의 길이와 너비에 따른 분산형(scatter) 그래프

plt.title('Sepal(꽃받침) Length vs Width', fontsize=16) # 타이틀
plt.xlabel('Sepal Length', fontsize=10)         # X축
plt.ylabel('Sepal Width', fontsize=12)          # Y축

# 붓꽃데이터를 점 그래프로 표현
plt.scatter(setosa['SepalLengthCm'], setosa['SepalWidthCm'],color='orange', label='Setosa')  
plt.scatter(versicolor['SepalLengthCm'], versicolor['SepalWidthCm'],color='blue', label='versicolor')  
plt.scatter(virginica['SepalLengthCm'], virginica['SepalWidthCm'],color='green', label='virginica')  

plt.legend()
plt.show()

- 꽃잎(Petal)의 길이와 너비에 따른 산포도 그래프 

In [ ]:
# 꽃잎(Petal)의 길이와 너비에 따른 분산형(scatter) 그래프
plt.rcParams["figure.figsize"] = (10,6)         # 그래프 크기 설정
plt.title('Petal(꽃잎) Length vs Width', fontsize=16) # 타이틀
plt.xlabel('Petal Length', fontsize=10)         # X축
plt.ylabel('Petal Width', fontsize=12)          # Y축

# 붓꽃데이터를 점 그래프로 표현
plt.scatter(setosa['PetalLengthCm'], setosa['PetalWidthCm'],color='orange', label='Setosa')  
plt.scatter(versicolor['PetalLengthCm'], versicolor['PetalWidthCm'],color='blue', label='versicolor')  
plt.scatter(virginica['PetalLengthCm'], virginica['PetalWidthCm'],color='green', label='virginica')  

plt.legend()
plt.show()

### [실습] K-NN 알고리즘 직접 만들기

In [ ]:
# 데이터 불러오기
import pandas as pd  
import numpy as np

iris = pd.read_csv('data/Iris.csv')        # 붖꽃 데이터 불러오기

# 계산의 편의를 위해 데이터셋의 형식을 numpy로 변환
xy = np.array(iris)

#------------------------------------
# Feature와 Target 데이터 분류
#------------------------------------
# 테이블의 1~4 열벡터를 features(특성)에 저장
features = xy[:, 1:-1]

# 테이블의 마지막 열벡터를 target_values(어떤종류)에 저장
target_value = xy[:, [-1]]


# 유클리드 거리법: Distance 함수 만들기
def Distance(A, B):
    return np.sqrt(np.sum(np.power((A-B),2)))

#------------------------------------
# 붓꽃 분류기 함수 작성하기
#------------------------------------
# 마지막 행벡터와 거리가 가까운 K개의 데이터의 라벨 중 빈도가 가장 높은 라벨을 반환하는 함수 정의하기
def K_NN(Unknown,features,K):
    # ① 데이터 간의 거리 계산
    distance_result = np.zeros(150)
    for i in range(len(features)):   
        distance_result[i]= Distance(Unknown,features[i])    
        
    # ② 분류하려는 데이터와 가까운 순서대로 나열
    index = distance_result.argsort()    
    
    # ③ K개의 라벨 확인
    target_result = []
    result=[0,0,0]
    for i in range(K):
        target_result.append(target_value[index[i]])
        if target_result[i]=='Iris-setosa':
           result[0]+=1
        elif target_result[i]=='Iris-versicolor':
             result[1]+=1
        else:
             result[2]+=1
    # ④ 라벨의 빈도가 가장 높은 값 반환
    max_label=result.index(max(result))
    species = {0:"setosa", 1:"versicolor", 2:"virginica"}
    species_result = species[max_label]
    return species_result


#------------------------------------
# 붓꽃 분류 함수를 사용하여 가상의 데이터 분류하기
#------------------------------------
# 마지막 150번째 데이터(인덱스 번호는 0부터 이므로 149)
test_1 = features[2]

# 150번째와 유사한 가상의 데이터 
# test_2 = np.array([6,2.9,5,2])
test_2 = np.array([5.8, 2.7, 5.1, 1.9])

# K_NN 분류기 함수를 이용하여 분류하기
result_1=K_NN(test_1,features,5)
result_2=K_NN(test_2,features,5)

#------------------------------------
# 결과 출력
#------------------------------------
print(f"실제 데이터를 분류한 결과 : {result_1}")
print(f"가상 데이터를 분류한 결과 : {result_2}")

### [실습] Scikit-learn의 K-NN 사용하여 붓꽃  분류하기

#### 1. Scikit-learn으로 붓꽃 데이터 가져오기

In [ ]:
from sklearn.datasets import load_iris
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split  

import pandas as pd
import numpy as np

# 붓꽃 데이터 셋트를 로딩
iris = load_iris()    #iris의 품종 : [0:'setosa',1:'versicolor',2:'virginica']
print(iris)

# X 피쳐만 추출: Features
iris_data = iris.data   
print(type(iris.data))
print(iris.data.shape) # (150, 4)

# Y 답(label)만 추출: target_Value
iris_label = iris.target

# Data Frame 만들기
iris_df = pd.DataFrame(data=iris_data,columns=iris.feature_names)
iris_df['label'] = iris_label  # # 예측(predict) 수행       
iris_df

#### 2.학습데이터 분류하고 K-NN알고리즘 적용하여 학습모델(예측모델) 만들기

In [ ]:
# train(학습데이터)와 test(검증데이터) 세트로 분리 :  80%:20% 비율, 120개(train), 30개(test)
x_train,x_test,y_train,y_test = train_test_split(iris_data,iris_label,test_size=0.2,  # test_size:0.2(20%)
                                                  random_state=11) # random seed 고정


# K-NN 알고리즘 클래스 사용해서 학습데이터 fit하기 = 학습모델 만들기
dt_clf = KNeighborsClassifier(n_neighbors=5)
dt_clf.fit(x_train, y_train)
           
           
# 테스트테이터로 예측(predict) 수행하기
pred = dt_clf.predict(x_test) 
print('예측:', pred)
print('실제:', y_test)    # 예측된 결과와 테스트데이터 출력해서 비교하기 

#### 3. 모델 정확도 측정하기

In [ ]:
# 정확도 측정 : accuracy
from sklearn.metrics import accuracy_score, classification_report 

# 정확도  : 1회만 예측한 결과의 정확도
#print('정확도: {0:.4f}'.format(accuracy_score(y_test, pred)))
print(f'정확도: {accuracy_score(y_test, pred):.4f}')

#### 4. 교차 검증하기(cross validation)

In [ ]:
# 테스트 데이터에만 과적합 될 수 있으므로 데이터를 여러개로 나누어 테스트를 여러번 수행하여 평균 정확도를 구함
from sklearn.model_selection import cross_val_score

cv_score = cross_val_score(dt_clf, iris.data, iris.target,  # Estimator,X,Y값
                           scoring='accuracy', cv = 3) # cv = 3, 3개로 쪼개어 검증(predict를 3회 수행)

# 내부적으로 Stratified K-Fold 사용됨, 평가지표를 한개만 구할 수 있어서 StatifiedKFold사용 권장
print('교차 검증 정확도:',cv_score)  # [0.98 0.92 0.98]
print('교차 검증 평균 정확도:',np.round(np.mean(cv_score),4)) # 0.96  : 3회 예측 한 결과의 정확도들의 평균

#### 5. 학습모델에 넣어  테스트데이터(고흐의 붓꽃 데이터) 예측하기

In [ ]:
# 그림에서 수집한 데이터 
X_test = np.array([[2.7, 2.4, 1.65, 0.67],
                   [5.84, 5.48, 3, 2.16],
                   [5.8, 2.7, 5.1, 1.9]]) 

# K-NN을 이용한 예측(predict) 수행
pred = dt_clf.predict(X_test)
print(pred)

# 분류 결과 출력하기
print('고흐의 붓꽃데이터 결과:')
names = {0:'setosa',1:'versicolor',2:'virginica'}
for i in pred:
    print(f' {X_test[i]}: {i} -> {names[i]}')

----------------------

THE END